<a href="https://colab.research.google.com/github/fedjkeee/NLP_-Deeplearning/blob/main/Tweets_Seq_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<a href="https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/TensorFlow%20In%20Practice/Course%203%20-%20NLP/Course%203%20-%20Week%202%20-%20Lesson%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
print(tf.__version__)

# !pip install -q tensorflow-datasets

2.4.1


In [4]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
columns=['id','tdate','tmane','ttext','ttype','terp','trtw','tfav','tstcount','tfol','tfrien','listcount']

neg = pd.read_csv('https://raw.githubusercontent.com/tixonsit/mmdad_materials/master/negative.csv', encoding='utf8', sep=';', header=None)
pos = pd.read_csv('https://raw.githubusercontent.com/tixonsit/mmdad_materials/master/positive.csv', encoding='utf8', sep=';', header=None)

In [6]:

df = pos.append(neg, ignore_index = True)
df = df.sample(frac=1).reset_index(drop=True)
len(df)

226834

In [7]:
# сделаем привычные 1, 0
df[4] = df[4].map({-1:0, 1:1})
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,410481587319750656,1386701411,wombat5301,RT @fillip_ina: @Z8SS @wombat5301 @PanovaOlga1...,1,0,1,0,3124,1096,1248,33
1,424583332085919744,1390063529,Daniela_Sk,@mikkel_ru в больницу положили за день до :(,0,0,0,0,23290,696,247,6
2,409577541037326336,1386485870,NoorySan,@mauerburo59 А кроме комнаты ты нигде не бывае...,1,0,0,0,12133,538,354,6
3,409201101565870080,1386396119,tyfuakova,RT @SashaPaches: @tyfuakova с Днем Рождения) в...,1,0,1,0,1823,64,45,0
4,410693900492615680,1386752030,kirillvip,RT @skatnerpro: @kirillvip @kozzlou он просто ...,1,0,1,0,19539,205,94,3


In [8]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2, shuffle=True)

In [9]:
df_train.shape, df_test.shape

((181467, 12), (45367, 12))

In [16]:
df_train[[3,4]]

,3,4
36981,@Julia6_9Styles вышел)\nТам плакат ван ли из m...,1
220763,@Rush_bint_Ahma1 самая жесть ((((( особенно ко...,0
204008,@technoslav так никто не говорит что стрелять ...,1
30302,@Kseniya__Sh ну и нафиг тогда тебе какой-то За...,1
123067,"@realritaskeeter Я знааю,но мне все равно стр...",1
...,...,...
194881,@Ksyunchik190498 и не заходишь мартышка( и чае...,0
22039,@_Stay_SStrong_ 2:(\nНО ВСЁ РАВНО\nЭТО ПРЕКРАСНО,0
39456,"Подарите мне книгу ""Ёжик в тумане"" Сергея Козл...",0
33355,"Это win! Не поленитесь, послушайте дядьку на в...",1


In [18]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete62G7W4/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete62G7W4/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete62G7W4/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [28]:
import numpy as np

#train_data, test_data = imdb['train'], imdb['test']
train_data, test_data = dataset_train,dataset_test
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


In [27]:
dataset_train = tf.data.Dataset.from_tensor_slices((df_train[3].values, df_train[4].values))
dataset_test = tf.data.Dataset.from_tensor_slices((df_train[3].values, df_train[4].values))

In [29]:
vocab_size = 30000 #10000
embedding_dim = 16
max_length = 140
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)


In [30]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? kseniya sh ну и нафиг тогда тебе какой то <OOV> дом2 <OOV> зла
@Kseniya__Sh ну и нафиг тогда тебе какой-то Задойнов?)#дом2_цитадель_зла


In [31]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 140, 16)           480000    
_________________________________________________________________
flatten (Flatten)            (None, 2240)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 13446     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 493,453
Trainable params: 493,453
Non-trainable params: 0
_________________________________________________________________


In [32]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
5671/5671 [==============================] - 46s 8ms/step - loss: 0.5356 - accuracy: 0.7161 - val_loss: 0.3786 - val_accuracy: 0.8363
Epoch 2/10
5671/5671 [==============================] - 43s 8ms/step - loss: 0.3720 - accuracy: 0.8335 - val_loss: 0.2859 - val_accuracy: 0.8875
Epoch 3/10
5671/5671 [==============================] - 44s 8ms/step - loss: 0.2796 - accuracy: 0.8837 - val_loss: 0.1888 - val_accuracy: 0.9358
Epoch 4/10
5671/5671 [==============================] - 43s 8ms/step - loss: 0.1887 - accuracy: 0.9290 - val_loss: 0.1185 - val_accuracy: 0.9621
Epoch 5/10
5671/5671 [==============================] - 44s 8ms/step - loss: 0.1243 - accuracy: 0.9557 - val_loss: 0.0789 - val_accuracy: 0.9753
Epoch 6/10
5671/5671 [==============================] - 43s 8ms/step - loss: 0.0873 - accuracy: 0.9690 - val_loss: 0.0574 - val_accuracy: 0.9820
Epoch 7/10
5671/5671 [==============================] - 43s 8ms/step - loss: 0.0655 - accuracy: 0.9777 - val_loss: 0.0445 - val_ac

In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [ ]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [ ]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 64, 102, 12, 7, 478, 1200]]


In [33]:
test_review_bad = "Сбер инвестор самое лютое говнище в которое я влез. Крайне не рекомендую! Уебищное приложение, деньги нихуя не выведешь. Чмошники блять #сбер #инвестор"
test_review_good = "Пришло оповещение от Сбербанка, что мол, средства которые я переводил будут мне возвращены в ближайшее время. Неплохое завершение истории. Буду ждать.Спасибо #сбербанк !"
sequences = tokenizer.texts_to_sequences([test_review_bad,test_review_good])
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

In [34]:
model.predict(padded)

array([[0.16850194],
       [0.9974987 ]], dtype=float32)

In [36]:
import tensorflow_hub as hub
bert_layer = hub.KerasLayer("/content/drive/MyDrive/rubert_tf20",
                            trainable=True)

In [ ]:
model_2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model2.summary()